In [40]:
import spacy
import nltk
import re
import string
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import json
%matplotlib inline
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer



In [89]:
newline = []

textstring = ''
with open('Hansard_preCovid.csv', 'r', encoding='utf-16-le') as inp, open('newtextfile.txt', 'w') as out:
    for line in inp:
      newline.append(re.findall(r'"(.*)"', line))
      
temp = pd.Series(newline)

initialDF_preCovid = pd.DataFrame({'Text': temp})
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also', '(', ')', ',', '\'', ':', '.', '?', ']', '[', '-', 'mr.']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].astype(str).str.lower()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['Text'].apply(word_tokenize)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?


In [90]:
initialDF_midCovid= pd.read_csv('Hansard_mid_covid.csv', sep=',', encoding='utf-16-le')
initialDF_midCovid.drop(['Publication', 'First Name', 'Last Name','Constituency', 'Province', 'Date', 'Time', 'Page'], axis=1)
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also', '(', ')', ',', '\'', ':', '.', '?', ']', '[', '-', 'mr.']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()

initialDF_midCovid['Text'] = initialDF_midCovid['Text'].astype(str).str.lower()
initialDF_midCovid['Text'] = initialDF_midCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_midCovid['text_tokens'] = initialDF_midCovid['Text'].apply(word_tokenize)
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?

In [91]:


bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents(initialDF_midCovid['text_tokens'])
finder.apply_freq_filter(15)
bigram_scores = finder.score_ngrams(bigram_measures.pmi)

In [92]:


trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_documents(initialDF_midCovid['text_tokens'])
finder.apply_freq_filter(15)
trigram_scores = finder.score_ngrams(trigram_measures.pmi)

In [93]:
bigram_pmi = pd.DataFrame(bigram_scores)
bigram_pmi.columns = ['bigram', 'pmi']
bigram_pmi.sort_values(by='pmi', axis = 0, ascending = True, inplace = True)

In [94]:
trigram_pmi = pd.DataFrame(trigram_scores)
trigram_pmi.columns = ['trigram', 'pmi']
trigram_pmi.sort_values(by='pmi', axis = 0, ascending = True, inplace = True)

In [95]:
# Filter for bigrams with only noun-type structures
def bigram_filter(bigram):
    tag = nltk.pos_tag(bigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['NN']:
        return False
    if bigram[0] in clearWords or bigram[1] in clearWords:
        return False
    if 'n' in bigram or 't' in bigram:
        return False
    if 'PRON' in bigram:
        return False
    return True

In [96]:
# Filter for trigrams with only noun-type structures
def trigram_filter(trigram):
    tag = nltk.pos_tag(trigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['JJ','NN']:
        return False
    if trigram[0] in clearWords or trigram[-1] in clearWords or trigram[1] in clearWords:
        return False
    if 'n' in trigram or 't' in trigram:
         return False
    if 'PRON' in trigram:
        return False
    return True 

In [97]:
# Can set pmi threshold to whatever makes sense - eyeball through and select threshold where n-grams stop making sense
# choose top 500 ngrams in this case ranked by PMI that have noun like structures
filtered_bigram = bigram_pmi[bigram_pmi.apply(lambda bigram:\
                                              bigram_filter(bigram['bigram'])\
                                              and bigram.pmi > 7.7, axis = 1)][:200]

filtered_trigram = trigram_pmi[trigram_pmi.apply(lambda trigram: \
                                                 trigram_filter(trigram['trigram'])\
                                                 and trigram.pmi >7.7, axis = 1)][:200]


bigrams = [' '.join(x) for x in filtered_bigram.bigram.values if len(x[0]) > 2 or len(x[1]) > 2]
trigrams = [' '.join(x) for x in filtered_trigram.trigram.values if len(x[0]) > 2 or len(x[1]) > 2 and len(x[2]) > 2]

In [99]:
trigrams[:200]

['foreign worker program',
 'canada emergency response',
 'temporary foreign worker',
 'emergency response benefit',
 'since january 1',
 'return tabled question',
 'immigration refugee citizenship']

In [87]:
trigram_pmi.head(120)

,trigram,pmi
6,"(foreign, worker, program)",10.669958
5,"(canada, emergency, response)",13.569155
4,"(temporary, foreign, worker)",14.120216
3,"(emergency, response, benefit)",16.239618
2,"(since, january, 1)",18.932595
1,"(return, tabled, question)",18.970362
0,"(immigration, refugee, citizenship)",19.912229


In [100]:


# Concatenate n-grams
def replace_ngram(x):
    for gram in trigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    for gram in bigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    return x



In [109]:
midcovid_ngrams = initialDF_midCovid['text_tokens']

In [120]:
initialDF_midCovid['text_tokens'].apply(lambda x: replace_ngram(str(x)))

0      ['honour', 'present', 'official', 'language', ...
1      ['man', "'s", 'name', 'etched', 'heart', 'name...
2      ['would', 'take', 'opportunity', 'thank', 'pub...
3      ['end', 'day', 'important', 'recognize', 'prog...
4      ['pleased', 'rise', 'speak', 'bill', 'c‑206', ...
                             ...                        
236    ['pleased', 'rise', 'house', 'today', 'speak',...
237    ['would', 'begin', 'acknowledging', 'gathered'...
238    ['pleased', 'opportunity', 'discus', 'benefit'...
239    ['pleasure', 'rise', 'house', 'today', 'first'...
240    ['today', 'making', 'heartfelt', 'plea', 'hous...
Name: text_tokens, Length: 241, dtype: object

In [124]:
initialDF_midCovid['text_tokens'][6]

['wednesday',
 'federal',
 'assistance',
 'farmer',
 'receiving',
 'cover',
 'cost',
 'quarantine',
 'temporary',
 'foreign',
 'worker',
 'cut',
 'half.ottawa',
 'giving',
 'farmer',
 'reimbursement',
 '$',
 '1,500',
 'apologize',
 'forcing',
 'cover',
 'cost',
 'quarantine',
 'federal',
 'government',
 'cut',
 'amount',
 '$',
 '750',
 'farmer',
 'still',
 'need',
 'bring',
 'worker',
 'quarantine',
 'still',
 'mandatory',
 'health',
 'measure',
 'changed',
 'cost',
 'changed',
 'either',
 'thing',
 'changed',
 'ottawa',
 'longer',
 'part.will',
 'minister',
 'immigration',
 'refugee',
 'citizenship',
 'immediately',
 'reverse',
 'cut']